In [1]:
from __future__ import print_function
from tensorflow.keras import backend as K
from tensorflow.keras import layers, Sequential, regularizers
from tensorflow.keras.layers import Layer
from tensorflow.keras import activations
from tensorflow.keras import utils
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

from tensorflow.keras.callbacks import LearningRateScheduler

#import keras
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from numpy import interp

import math
from itertools import cycle
import numpy as np
import pandas as pd
import random
import seaborn as sn
from matplotlib import pyplot as plt

from evaluation import compute_performance_measures

from utils import *
from layers import *

K.set_image_data_format('channels_last')

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  #按需分配显存
session = tf.compat.v1.Session(config=config)

# tf.random.set_seed(111111)
# np.random.seed(111111)
# random.seed(111111)

tf.__version__

'2.3.0'

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from tensorflow.keras.applications.densenet import DenseNet121

def create_model(inputs):
    model_densenet = DenseNet121(include_top=False, weights=None, input_shape=(128, 128, 3))
    model_densenet.load_weights('densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5')
    for layers in model_densenet.layers:
        layers.trainable = True

    x = GlobalAveragePooling2D(name='final_pooling')(model_densenet.output)
    x = Dense(3, activation='softmax', name='fc')(x)
    model = Model(inputs=model_densenet.input, outputs=x, name='densenet121_pretrain')
    return model

In [4]:
inputs = Input(shape=(128, 128, 3))
model = create_model(inputs)
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "densenet121_pretrain"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
_______________________________________________________________________________

In [7]:
#learning decay rate schedule
def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.5 
    epochs_drop = 20  
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate

In [8]:
batch_size = 16  
num_classes = 3
epochs = 100

images= np.load("data/image.npy")
labels= np.load("data/label.npy")

In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)
cvscores = []
cvpre = []
cvrecall  =[]
cvf1 = []
cvauc = []

for k, (train, test) in enumerate(kfold.split(images, labels)):
    
    path = "model/"
    
    x_train = images[train]
    x_test = images[test]
    y_train = labels[train]
    y_test = labels[test]
    
#     np.save(path + 'data/x_train.npy', x_train)
#     np.save(path + 'data/y_train.npy', y_train)
#     np.save(path + 'data/x_test.npy', x_test)
#     np.save(path + 'data/y_test.npy', y_test)
    
    #class weights to handle class imbalance
    class_weights = {0: 1-np.count_nonzero(y_train==0)/len(y_train), 
                     1: 1-np.count_nonzero(y_train==1)/len(y_train), 
                     2: 1-np.count_nonzero(y_train==2)/len(y_train)}

    # 将整型标签转为onehot
    y_train = utils.to_categorical(y_train, num_classes)
    y_test = utils.to_categorical(y_test, num_classes)
    
    
    # The best model is selected based on the loss value on the validation set
    filepath = path+"denseNet121-weights-best.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    
    # learning schedule callback
    lrate = LearningRateScheduler(step_decay)

    callbacks_list = [checkpoint, lrate]
    

    inputs = Input(shape=(128, 128, 3))
    #===================================Model======================================
    model = create_model(inputs)   
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    
    print("========================= 第"+ str(k+1) +"折开始 ============================")
    history = model.fit(x_train, y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=2,
                    #validation_split=0.1,
                    validation_data=(x_test, y_test),
                    class_weight=class_weights, 
                    shuffle=True, 
                    callbacks=callbacks_list)
    
#     np.save(path+'acc.npy', history.history['accuracy'])
#     np.save(path+'val_acc.npy', history.history['val_accuracy'])
#     np.save(path+'loss.npy', history.history['loss'])
#     np.save(path+'val_loss.npy', history.history['val_loss'])
    
    model.load_weights(filepath)
    # evaluate the model
    scores = model.evaluate(x_test, y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
    predict = model.predict([x_test])
    
    # ===================auc=========================
    n_classes = y_test.shape[1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], predict[:, i], )
        roc_auc[i] = auc(fpr[i], tpr[i])
    roc_auc["macro"] = roc_auc_score(y_test, predict, multi_class="ovo", average="macro")
    roc_auc["weighted"] = roc_auc_score(y_test, predict, multi_class="ovo", average="weighted")
    # ===================================================

    y_pre = np.argmax(predict, axis=1)
    y_test = np.argmax(y_test, axis=1)

    report = classification_report(y_test, y_pre, output_dict=True)
    df1 = pd.DataFrame(report).transpose()
    
    df1['auc'] = [roc_auc[0], roc_auc[1], roc_auc[2], " ", roc_auc["macro"], roc_auc["weighted"]]
    
    # Write it into csv format
    #df1.to_csv(path+'report.csv', index=True, header=True)
    
    cvpre.append(df1.loc['macro avg','precision'] * 100)
    cvrecall.append(df1.loc['macro avg','recall'] * 100)
    cvf1.append(df1.loc['macro avg','f1-score'] * 100)
    cvauc.append(df1.loc['macro avg','auc'] * 100)
    
#     data = confusion_matrix(y_test, y_pre)
#     names = ['normal', 'pneumonia', 'COVID-19']
#     df_cm = pd.DataFrame(data, columns=names, index=names)
#     df_cm.to_csv(path+'cm.csv', index=True, header=True)
    print("========================= 第"+ str(k+1) +"折结束 ============================")
    
print("accuracy：%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("precision：%.2f%% (+/- %.2f%%)" % (np.mean(cvpre), np.std(cvpre)))
print("recall：%.2f%% (+/- %.2f%%)" % (np.mean(cvrecall), np.std(cvrecall)))
print("f1-score：%.2f%% (+/- %.2f%%)" % (np.mean(cvf1), np.std(cvf1)))
print("auc：%.2f%% (+/- %.2f%%)" % (np.mean(cvauc), np.std(cvauc)))

========================= 第1折开始 ============================
Epoch 1/100

Epoch 00001: val_accuracy improved from -inf to 0.50251, saving model to model\denseNet121-weights-best.h5
50/50 - 11s - loss: 1.0817 - accuracy: 0.3975 - val_loss: 2.0566 - val_accuracy: 0.5025
Epoch 2/100

Epoch 00002: val_accuracy improved from 0.50251 to 0.51256, saving model to model\denseNet121-weights-best.h5
50/50 - 6s - loss: 0.8187 - accuracy: 0.4692 - val_loss: 1.6121 - val_accuracy: 0.5126
Epoch 3/100

Epoch 00003: val_accuracy did not improve from 0.51256
50/50 - 5s - loss: 0.6254 - accuracy: 0.5799 - val_loss: 1.2778 - val_accuracy: 0.5075
Epoch 4/100

Epoch 00004: val_accuracy did not improve from 0.51256
50/50 - 5s - loss: 0.5201 - accuracy: 0.6377 - val_loss: 1.1802 - val_accuracy: 0.4874
Epoch 5/100

Epoch 00005: val_accuracy did not improve from 0.51256
50/50 - 5s - loss: 0.4453 - accuracy: 0.7283 - val_loss: 1.1176 - val_accuracy: 0.5075
Epoch 6/100

Epoch 00006: val_accuracy improved from 0.5

Epoch 48/100

Epoch 00048: val_accuracy did not improve from 0.93467
50/50 - 5s - loss: 0.1258 - accuracy: 0.9648 - val_loss: 0.2556 - val_accuracy: 0.9246
Epoch 49/100

Epoch 00049: val_accuracy did not improve from 0.93467
50/50 - 5s - loss: 0.1301 - accuracy: 0.9572 - val_loss: 0.2579 - val_accuracy: 0.9246
Epoch 50/100

Epoch 00050: val_accuracy did not improve from 0.93467
50/50 - 5s - loss: 0.1281 - accuracy: 0.9547 - val_loss: 0.2522 - val_accuracy: 0.9246
Epoch 51/100

Epoch 00051: val_accuracy did not improve from 0.93467
50/50 - 5s - loss: 0.1202 - accuracy: 0.9686 - val_loss: 0.2434 - val_accuracy: 0.9246
Epoch 52/100

Epoch 00052: val_accuracy did not improve from 0.93467
50/50 - 5s - loss: 0.1188 - accuracy: 0.9635 - val_loss: 0.2477 - val_accuracy: 0.9347
Epoch 53/100

Epoch 00053: val_accuracy did not improve from 0.93467
50/50 - 5s - loss: 0.1192 - accuracy: 0.9635 - val_loss: 0.2531 - val_accuracy: 0.9246
Epoch 54/100

Epoch 00054: val_accuracy improved from 0.93467 to

accuracy: 94.47%
========================= 第1折结束 ============================
========================= 第2折开始 ============================
Epoch 1/100

Epoch 00001: val_accuracy improved from -inf to 0.26633, saving model to model\denseNet121-weights-best.h5
50/50 - 7s - loss: 1.0404 - accuracy: 0.3660 - val_loss: 1.6350 - val_accuracy: 0.2663
Epoch 2/100

Epoch 00002: val_accuracy improved from 0.26633 to 0.36683, saving model to model\denseNet121-weights-best.h5
50/50 - 6s - loss: 0.7796 - accuracy: 0.4591 - val_loss: 1.4190 - val_accuracy: 0.3668
Epoch 3/100

Epoch 00003: val_accuracy improved from 0.36683 to 0.39196, saving model to model\denseNet121-weights-best.h5
50/50 - 6s - loss: 0.6265 - accuracy: 0.5535 - val_loss: 1.3191 - val_accuracy: 0.3920
Epoch 4/100

Epoch 00004: val_accuracy improved from 0.39196 to 0.41709, saving model to model\denseNet121-weights-best.h5
50/50 - 6s - loss: 0.5473 - accuracy: 0.6377 - val_loss: 1.2305 - val_accuracy: 0.4171
Epoch 5/100

Epoch 00005


Epoch 00046: val_accuracy did not improve from 0.93970
50/50 - 5s - loss: 0.1405 - accuracy: 0.9472 - val_loss: 0.2387 - val_accuracy: 0.9296
Epoch 47/100

Epoch 00047: val_accuracy did not improve from 0.93970
50/50 - 5s - loss: 0.1341 - accuracy: 0.9547 - val_loss: 0.2377 - val_accuracy: 0.9347
Epoch 48/100

Epoch 00048: val_accuracy did not improve from 0.93970
50/50 - 5s - loss: 0.1350 - accuracy: 0.9522 - val_loss: 0.2444 - val_accuracy: 0.9296
Epoch 49/100

Epoch 00049: val_accuracy did not improve from 0.93970
50/50 - 5s - loss: 0.1317 - accuracy: 0.9597 - val_loss: 0.2413 - val_accuracy: 0.9347
Epoch 50/100

Epoch 00050: val_accuracy did not improve from 0.93970
50/50 - 5s - loss: 0.1250 - accuracy: 0.9660 - val_loss: 0.2355 - val_accuracy: 0.9347
Epoch 51/100

Epoch 00051: val_accuracy did not improve from 0.93970
50/50 - 5s - loss: 0.1291 - accuracy: 0.9585 - val_loss: 0.2387 - val_accuracy: 0.9397
Epoch 52/100

Epoch 00052: val_accuracy did not improve from 0.93970
50/50 - 

Epoch 99/100

Epoch 00099: val_accuracy did not improve from 0.93970
50/50 - 5s - loss: 0.1199 - accuracy: 0.9610 - val_loss: 0.2192 - val_accuracy: 0.9397
Epoch 100/100

Epoch 00100: val_accuracy did not improve from 0.93970
50/50 - 5s - loss: 0.1069 - accuracy: 0.9811 - val_loss: 0.2189 - val_accuracy: 0.9397
accuracy: 93.97%
========================= 第2折结束 ============================
========================= 第3折开始 ============================
Epoch 1/100

Epoch 00001: val_accuracy improved from -inf to 0.27136, saving model to model\denseNet121-weights-best.h5
50/50 - 7s - loss: 1.1025 - accuracy: 0.3006 - val_loss: 1.8534 - val_accuracy: 0.2714
Epoch 2/100

Epoch 00002: val_accuracy improved from 0.27136 to 0.29146, saving model to model\denseNet121-weights-best.h5
50/50 - 6s - loss: 0.8196 - accuracy: 0.3811 - val_loss: 1.6298 - val_accuracy: 0.2915
Epoch 3/100

Epoch 00003: val_accuracy improved from 0.29146 to 0.33668, saving model to model\denseNet121-weights-best.h5
50/50 - 

...

Epoch 95/100

Epoch 00095: val_accuracy did not improve from 0.90955
50/50 - 5s - loss: 0.1026 - accuracy: 0.9723 - val_loss: 0.2978 - val_accuracy: 0.8995
Epoch 96/100

Epoch 00096: val_accuracy did not improve from 0.90955
50/50 - 5s - loss: 0.1191 - accuracy: 0.9698 - val_loss: 0.2982 - val_accuracy: 0.8995
Epoch 97/100

Epoch 00097: val_accuracy did not improve from 0.90955
50/50 - 5s - loss: 0.1139 - accuracy: 0.9610 - val_loss: 0.2989 - val_accuracy: 0.9045
Epoch 98/100

Epoch 00098: val_accuracy did not improve from 0.90955
50/50 - 5s - loss: 0.1049 - accuracy: 0.9698 - val_loss: 0.2981 - val_accuracy: 0.9045
Epoch 99/100

Epoch 00099: val_accuracy did not improve from 0.90955
50/50 - 5s - loss: 0.1126 - accuracy: 0.9698 - val_loss: 0.2969 - val_accuracy: 0.8995
Epoch 100/100

Epoch 00100: val_accuracy did not improve from 0.90955
50/50 - 5s - loss: 0.1075 - accuracy: 0.9635 - val_loss: 0.2961 - val_accuracy: 0.8945
accuracy: 90.95%
========================= 第3折结束 ==============

Epoch 41/100

Epoch 00041: val_accuracy did not improve from 0.90452
50/50 - 5s - loss: 0.1246 - accuracy: 0.9610 - val_loss: 0.3087 - val_accuracy: 0.9045
Epoch 42/100

Epoch 00042: val_accuracy did not improve from 0.90452
50/50 - 5s - loss: 0.1402 - accuracy: 0.9421 - val_loss: 0.2990 - val_accuracy: 0.9045
Epoch 43/100

Epoch 00043: val_accuracy did not improve from 0.90452
50/50 - 5s - loss: 0.1210 - accuracy: 0.9572 - val_loss: 0.3091 - val_accuracy: 0.8945
Epoch 44/100

Epoch 00044: val_accuracy did not improve from 0.90452
50/50 - 5s - loss: 0.1376 - accuracy: 0.9522 - val_loss: 0.3071 - val_accuracy: 0.8945
Epoch 45/100

Epoch 00045: val_accuracy did not improve from 0.90452
50/50 - 5s - loss: 0.1273 - accuracy: 0.9560 - val_loss: 0.3116 - val_accuracy: 0.8894
Epoch 46/100

Epoch 00046: val_accuracy did not improve from 0.90452
50/50 - 5s - loss: 0.1264 - accuracy: 0.9547 - val_loss: 0.3126 - val_accuracy: 0.8945
Epoch 47/100

Epoch 00047: val_accuracy did not improve from 0.9

Epoch 93/100

Epoch 00093: val_accuracy did not improve from 0.91457
50/50 - 5s - loss: 0.1036 - accuracy: 0.9686 - val_loss: 0.2773 - val_accuracy: 0.9045
Epoch 94/100

Epoch 00094: val_accuracy did not improve from 0.91457
50/50 - 5s - loss: 0.1134 - accuracy: 0.9585 - val_loss: 0.2743 - val_accuracy: 0.9045
Epoch 95/100

Epoch 00095: val_accuracy did not improve from 0.91457
50/50 - 5s - loss: 0.1011 - accuracy: 0.9736 - val_loss: 0.2746 - val_accuracy: 0.9095
Epoch 96/100

Epoch 00096: val_accuracy did not improve from 0.91457
50/50 - 5s - loss: 0.1112 - accuracy: 0.9610 - val_loss: 0.2724 - val_accuracy: 0.9045
Epoch 97/100

Epoch 00097: val_accuracy did not improve from 0.91457
50/50 - 5s - loss: 0.1130 - accuracy: 0.9597 - val_loss: 0.2749 - val_accuracy: 0.9045
Epoch 98/100

Epoch 00098: val_accuracy did not improve from 0.91457
50/50 - 5s - loss: 0.0993 - accuracy: 0.9761 - val_loss: 0.2745 - val_accuracy: 0.9095
Epoch 99/100

Epoch 00099: val_accuracy did not improve from 0.9

Epoch 39/100

Epoch 00039: val_accuracy improved from 0.87879 to 0.88889, saving model to model\denseNet121-weights-best.h5
50/50 - 6s - loss: 0.1448 - accuracy: 0.9384 - val_loss: 0.3405 - val_accuracy: 0.8889
Epoch 40/100

Epoch 00040: val_accuracy did not improve from 0.88889
50/50 - 5s - loss: 0.1300 - accuracy: 0.9573 - val_loss: 0.3335 - val_accuracy: 0.8788
Epoch 41/100

Epoch 00041: val_accuracy did not improve from 0.88889
50/50 - 5s - loss: 0.1316 - accuracy: 0.9573 - val_loss: 0.3401 - val_accuracy: 0.8687
Epoch 42/100

Epoch 00042: val_accuracy did not improve from 0.88889
50/50 - 5s - loss: 0.1419 - accuracy: 0.9560 - val_loss: 0.3174 - val_accuracy: 0.8687
Epoch 43/100

Epoch 00043: val_accuracy did not improve from 0.88889
50/50 - 5s - loss: 0.1277 - accuracy: 0.9560 - val_loss: 0.3256 - val_accuracy: 0.8838
Epoch 44/100

Epoch 00044: val_accuracy did not improve from 0.88889
50/50 - 5s - loss: 0.1237 - accuracy: 0.9585 - val_loss: 0.3369 - val_accuracy: 0.8737
Epoch 45/

Epoch 92/100

Epoch 00092: val_accuracy did not improve from 0.88889
50/50 - 5s - loss: 0.1033 - accuracy: 0.9686 - val_loss: 0.3014 - val_accuracy: 0.8737
Epoch 93/100

Epoch 00093: val_accuracy did not improve from 0.88889
50/50 - 5s - loss: 0.1109 - accuracy: 0.9573 - val_loss: 0.2987 - val_accuracy: 0.8788
Epoch 94/100

Epoch 00094: val_accuracy did not improve from 0.88889
50/50 - 5s - loss: 0.1018 - accuracy: 0.9648 - val_loss: 0.2950 - val_accuracy: 0.8788
Epoch 95/100

Epoch 00095: val_accuracy did not improve from 0.88889
50/50 - 5s - loss: 0.1076 - accuracy: 0.9724 - val_loss: 0.2959 - val_accuracy: 0.8889
Epoch 96/100

Epoch 00096: val_accuracy did not improve from 0.88889
50/50 - 5s - loss: 0.1092 - accuracy: 0.9661 - val_loss: 0.2985 - val_accuracy: 0.8788
Epoch 97/100

Epoch 00097: val_accuracy did not improve from 0.88889
50/50 - 5s - loss: 0.0997 - accuracy: 0.9673 - val_loss: 0.2973 - val_accuracy: 0.8788
Epoch 98/100

Epoch 00098: val_accuracy did not improve from 0.8

In [10]:
X = np.load("newData/image.npy")
Y = np.load("newData/label.npy")

In [11]:
predict = model.predict([X])

y = label_binarize(Y, classes=[0, 1, 2])
n_classes = y.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y[:, i], predict[:, i], )
    roc_auc[i] = auc(fpr[i], tpr[i])
roc_auc["macro"] = roc_auc_score(y, predict, multi_class="ovo", average="macro")
roc_auc["weighted"] = roc_auc_score(y, predict, multi_class="ovo", average="weighted")


y_pre = np.argmax(predict, axis=1)
report = classification_report(Y, y_pre, output_dict=True)
df1 = pd.DataFrame(report).transpose()
df1['auc'] = [roc_auc[0], roc_auc[1], roc_auc[2], '', roc_auc["macro"], roc_auc["weighted"]]
df1

,precision,recall,f1-score,support,auc
0,0.889211,0.915735,0.902278,1341.000000,0.978168
1,0.871138,0.859480,0.865269,1345.000000,0.960135
2,0.932937,0.915833,0.924306,1200.000000,0.989029
accuracy,0.896294,0.896294,0.896294,0.896294,
macro avg,0.897762,0.897016,0.897284,3886.000000,0.975778
weighted avg,0.896458,0.896294,0.896271,3886.000000,0.975281
